In [ ]:
!pip install pystan~=2.14
!pip install fbprophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tqdm as tqdm

import matplotlib.pyplot as plt

%matplotlib inline 

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn


# special matplotlib argument for improved plots
from matplotlib import rcParams

In [ ]:
data = pd.read_csv("VND Historical Data.csv")
data.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2017/12/1,13928.5,13625.2,14105.5,13574.6,10420000,0.02
1,2017/12/4,14712.2,14054.9,14813.3,13979.1,11520000,0.06
2,2017/12/5,14257.2,14661.6,14712.2,14156.1,11100000,-0.03
3,2017/12/6,14130.8,14206.6,14257.2,13625.2,14660000,-0.01
4,2017/12/7,13701.0,14130.8,14156.1,13599.9,8850000,-0.03


In [ ]:
dataset = data.loc[:,["Date","Price"]]
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.Date = pd.to_datetime(dataset.Date,format = "%Y/%m/%d")
dataset.info()

In [ ]:
dataset = dataset.rename(columns = {"Price" : "y"})
dataset = dataset.rename(columns = {"Date" : "ds"})
dataset.head()

In [ ]:
test_days = 375
training_set = dataset.iloc[:-test_days, :]
test_set = dataset.iloc[-test_days:, :]
test_set.tail(5)

In [ ]:
print(len(training_set))
print(len(test_set))

In [ ]:
training_set.tail(5)

In [ ]:
m = Prophet(growth = "linear",
            yearly_seasonality = True,
            weekly_seasonality = True,
            daily_seasonality = False,
            seasonality_mode = "multiplicative",
            seasonality_prior_scale = 10,
            holidays_prior_scale = 10,
            changepoint_prior_scale = 0.05)
m.fit(training_set)

In [ ]:
future = m.make_future_dataframe(periods = len(test_set), freq = "D")
future.tail()

In [ ]:
future = pd.concat([future, dataset.iloc[:,1:]],
                   axis = 1)
future.tail(5)

In [ ]:
forecast = m.predict(future)
forecast.head()

In [ ]:
predictions_prophet = forecast.yhat[-test_days:].rename("prophet")
predictions_prophet[:5]

In [ ]:
from fbprophet.diagnostics import cross_validation
cs = cross_validation(m,
                         #Dự đoán 30 ngày tiếp theo
                         horizon = '30 days',
                         #Lấy 540 ngày dể đào tạo
                         initial = '365 days',
                         #Xử lý song song
                         parallel = "processes")

In [ ]:
y_true = cs['y'].values
y_pre = cs['yhat'].values

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
mae = mean_absolute_error(y_true,y_pre)
mape = mean_absolute_percentage_error(y_true,y_pre)
mse = mean_squared_error(y_true, y_pre)
rmse = np.sqrt(mse)
r2 = r2_score(y_true,y_pre)

print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape * 100:.2f}%")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2: {r2:.2f}")

In [ ]:
from sklearn.model_selection import ParameterGrid
param_grid = {'seasonality_prior_scale': np.arange(5, 10, 5), #[5, 10]  np.arange(5, 20, 5),
              'changepoint_prior_scale': np.arange(0.01, 0.05, 0.01),  # [0.01, 0.02, 0.03, 0.04, 0.05]
              'holidays_prior_scale': np.arange(5, 10, 5)}#[5, 10]
grid = ParameterGrid(param_grid)
list(grid)[:3]  

In [ ]:
rmse = []
for params in grid:
  #build model
  m = Prophet(growth = "linear",
            yearly_seasonality = True,
            weekly_seasonality = True,
            daily_seasonality = False,
            seasonality_mode = "multiplicative",
            seasonality_prior_scale = params['seasonality_prior_scale'],
            holidays_prior_scale = params['holidays_prior_scale'],
            changepoint_prior_scale = params['changepoint_prior_scale'])
  m.fit(training_set)

  #cross-validation
  cs = cross_validation(m,
                         horizon = '30 days',
                         initial = '540 days',
                         parallel = "processes")
  
  #gather the results
  error = np.sqrt(mean_squared_error(cs['y'], 
                                     cs['yhat']))
  rmse.append(error)

In [ ]:
best_params = grid[np.argmin(rmse)]
print(best_params)

In [ ]:
m = Prophet(growth = "linear",
          yearly_seasonality = True,
          weekly_seasonality = True,
          daily_seasonality = False,
          seasonality_mode = "multiplicative",
          seasonality_prior_scale = best_params['seasonality_prior_scale'],
          holidays_prior_scale = best_params['holidays_prior_scale'],
          changepoint_prior_scale = best_params['changepoint_prior_scale'])
m.fit(training_set)

In [ ]:
future = m.make_future_dataframe(periods = len(test_set),
                                 freq = "D")
future = pd.concat([future, dataset.iloc[:,1:]],
                   axis = 1)
future.tail(1)

In [ ]:
forecast = m.predict(future)
forecast.head()

In [ ]:
print(len(forecast['yhat']))
print(len(forecast.yhat[-test_days:]))
print(len(test_set["ds"]))

In [ ]:
predictions_prophet_tuned = forecast.yhat[-test_days:].rename("prophet_tuned")
predictions_prophet_tuned.index = test_set["ds"]
predictions_prophet_tuned.head()